In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4699/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4699/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_11_0,0.596998,-0.030350,-0.292639,0.347493,0.492022,6.889956,6.342078,0.251889,0.663947,0.701443,0.432813,0.707623,67.418464,158.001224,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_11_1,0.601210,-0.030742,-0.267932,0.348040,0.486880,6.892577,6.220855,0.251678,0.662343,0.697768,0.438741,0.703916,67.439476,158.022237,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_11_2,0.621593,-0.033584,-0.269672,0.349080,0.461995,6.911580,6.229393,0.251276,0.654379,0.679702,0.467427,0.685691,67.544403,158.127163,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_3_40_2,0.610010,-0.040586,-0.257243,-0.795645,0.476137,6.958400,6.168413,0.330546,1.080377,0.690026,0.316017,0.696106,99.484101,233.985776,"Hidden Size=[12], regularizer=0.02, learning_r..."
4,model_3_40_7,0.613289,-0.042448,-0.265215,-1.153801,0.472133,6.970856,6.207526,0.396476,1.113225,0.687119,0.321768,0.693174,99.500988,234.002663,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,model_2_22_4,0.626137,-5.986208,-2.019876,-0.119430,0.456447,46.716805,14.816423,0.722221,0.672979,0.675609,0.473822,0.681561,67.568566,158.151326,"Hidden Size=[8], regularizer=0.02, learning_ra..."
272,model_2_23_0,0.624726,-6.291244,-2.178431,0.548187,0.458170,48.756580,15.594339,0.511058,0.668329,0.676882,0.471837,0.682846,67.561031,158.143792,"Hidden Size=[8], regularizer=0.02, learning_ra..."
273,model_2_22_0,0.613705,-6.764402,-1.683168,-0.036994,0.471625,51.920592,13.164429,0.669036,0.662951,0.686750,0.456325,0.692801,67.503141,158.085901,"Hidden Size=[8], regularizer=0.02, learning_ra..."
274,model_2_23_7,0.601773,-8.446914,-6.084024,0.175247,0.486193,63.171555,34.756356,0.932900,0.635334,0.697276,0.439532,0.703419,67.442298,158.025058,"Hidden Size=[8], regularizer=0.02, learning_ra..."
